In [1]:
import os
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
from skimage import io
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
from imgaug import parameters as iap

# General

In [2]:
img_format = '.jpg'
labels_path = 'labels/'
path = 'images/'
cropped_images_path = 'cropped_images/'
cropped_labels_path = 'cropped_labels/'
new_labels = 'new_labels/'

## Remove Very Small Objects

In [68]:
threshold = 0.0001
for i in range(10):
    for item in os.listdir(labels_path):
        ann = np.loadtxt(labels_path+item)
        ann = ann.tolist()

        # multiple objects
        if type(ann[0]) == list:
            for obj in ann:
                if obj[3]*obj[4] < threshold:
                    ann.remove(obj)
            if ann != []:
                np.savetxt(labels_path+item, ann, fmt='%.10g')
            else:
                os.remove(path+item.replace(item[-4:], img_format))
                os.remove(labels_path+item)

        # single object    
        else:
            if ann[3]*ann[4] < threshold:
                os.remove(labels_path+item)
                os.remove(path+item.replace(item[-4:], img_format))

## Crop

In [ ]:
threshold = 0.0003
for i in range(5):
    for item in os.listdir(labels_path):
        ann = np.loadtxt(labels_path+item).tolist()
        width = 512
        height = 512
        i = 0
        margin = 30


        # multiple objects
        if type(ann[0]) == list:
            img_name = item.replace(item[-4:], '.jpg')
            image = imageio.imread(os.path.join(path,img_name))
            item = item.replace(item[-4:], '')
            ann_copy = ann.copy()
            
            h, w ,c = image.shape
            # specify bboxes
            bbs = BoundingBoxesOnImage([
                BoundingBox(x1=(obj[1]-obj[3]/2)*w, x2=(obj[1]+obj[3]/2)*w, y1=(obj[2]-obj[4]/2)*h, y2=(obj[2]+obj[4]/2)*h) for obj in ann]
                , shape=image.shape)


            for element in ann:
                annotation = []
                # names
                i += 1
                new_item_name = f'{item}_{i}'
                new_img_name = new_item_name + '.jpg'
                new_label_name = new_item_name +'.txt'

                if element[3]*element[4] < threshold:
                    # Transformations
                    seq = iaa.Sequential([
                    iaa.CropToFixedSize(width=width, height=height, position=((1-element[1]),(1-element[2])))
                    ])
                    # Output
                    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
                    # make sure object are completly inside (px)
                    for e in bbs_aug:
                        xmin=e[0][0]
                        ymin=e[0][1]
                        xmax=e[1][0]
                        ymax=e[1][1]

                        if (xmin > 0 - margin) and (width + margin > xmax) and (ymin > 0 - margin) and (height + margin > ymax):
                            if xmin < 0:
                                xmin = xmin + abs(xmin)
                            if xmax > width:
                                xmax = xmax - abs(width-xmax)
                            if ymin < 0:
                                ymin = ymin + abs(ymin)
                            if ymax > height:
                                ymax = ymax - abs(height-ymax)


                            x_center = (xmax+xmin)/(2*width)
                            y_center = (ymax+ymin)/(2*height)
                            obj_width = (xmax-xmin)/width
                            obj_height = (ymax-ymin)/height

                            annot = [int(0), x_center, y_center, obj_width, obj_height]
                            annotation.append(annot)

                    ann_copy.remove(element)
                    if annotation != []:
                        np.savetxt(os.path.join(cropped_labels_path, new_label_name), annotation, fmt='%.10g')
                        io.imsave(os.path.join(cropped_images_path, new_img_name), image_aug)
            
            if ann_copy != []:
                np.savetxt(os.path.join(labels_path, item+'.txt'), ann_copy, fmt='%.10g')
                io.imsave(os.path.join(path, img_name), image)
            if ann_copy == []:
                os.remove(os.path.join(labels_path, item+'.txt'))
                os.remove(os.path.join(path, img_name))



        # single object    
        else:
            if ann[3]*ann[4] < threshold:
                obj = ann

                # load image
                img_name = item.replace(item[-4:],'.jpg')
                image = io.imread(os.path.join(path, img_name))

                # extract h, w
                h, w , c = image.shape

                bbs = BoundingBoxesOnImage([
                    BoundingBox(x1=(obj[1]-obj[3]/2)*w, x2=(obj[1]+obj[3]/2)*w, y1=(obj[2]-obj[4]/2)*h, y2=(obj[2]+obj[4]/2)*h)]
                    , shape=image.shape)

                seq = iaa.Sequential([
                iaa.CropToFixedSize(width=width, height=height, position=((1-obj[1]),(1-obj[2])))
                ])

                # Output
                image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

                # make sure object are completly inside (px)
                e = bbs_aug
                xmin=e[0][0][0]
                ymin=e[0][0][1]
                xmax=e[0][1][0]
                ymax=e[0][1][1]

                if (xmin > 0 - margin) and (width + margin > xmax) and (ymin > 0 - margin) and (height + margin > ymax):
                    if xmin < 0:
                        xmin = xmin + abs(xmin)
                    if xmax > width:
                        xmax = xmax - abs(width-xmax)
                    if ymin < 0:
                        ymin = ymin + abs(ymin)
                    if ymax > height:
                        ymax = ymax - abs(height-ymax)

                    x_center = (xmax+xmin)/(2*width)
                    y_center = (ymax+ymin)/(2*height)
                    obj_width = (xmax-xmin)/width
                    obj_height = (ymax-ymin)/height

                    annot = [int(0), x_center, y_center, obj_width, obj_height]


                np.savetxt(os.path.join(cropped_labels_path, 'single-'+item), annot, newline=' ', fmt='%.10g')
                io.imsave(os.path.join(cropped_images_path, 'single-'+img_name), image_aug)
                os.remove(os.path.join(labels_path, item))
                os.remove(os.path.join(path, img_name))
print('Done')

# Bugs

* Needs more than 1 run
* not sure how position=() really works! (Some images are very similar)

# Remove Similiar Images

In [7]:
from skimage.metrics import structural_similarity as ssim

In [ ]:
threshold = .4
for im1 in os.listdir(cropped_images_path):
    for im2 in os.listdir(cropped_images_path):
        if im1 != im2:
            image1 = cv.imread(os.path.join(cropped_images_path,im1))
            image2 = cv.imread(os.path.join(cropped_images_path,im2))
            try:
                value = ssim(image1, image2, multichannel=True)
                if value > threshold:
                    os.remove(os.path.join(cropped_images_path,im2))
                    os.remove(os.path.join(cropped_labels_path,im2.replace(im2[-4:], '.txt')))
            except ValueError:
                pass

In [11]:
ssim(cv.imread('3691_1.jpg'), cv.imread('3691_2.jpg'), multichannel=True)

0.096686868795659